In [1]:
import os
from icecream import ic
import numpy as np
import time
import torch
import pandas as pd

import evaluate
rouge = evaluate.load('rouge')
bleu = evaluate.load('bleu')
meteor = evaluate.load('meteor')
from evaluate import load
bertscore = load("bertscore")

Using the latest cached version of the module from /home2/rudra.dhar/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--bleu/9e0985c1200e367cce45605ce0ecb5ede079894e0f24f54613fca08eeb8aff76 (last modified on Tue Aug 15 17:26:46 2023) since it couldn't be found locally at evaluate-metric--bleu, or remotely on the Hugging Face Hub.
[nltk_data] Error loading wordnet: <urlopen error [Errno 104]
[nltk_data]     Connection reset by peer>
[nltk_data] Error loading punkt: <urlopen error [Errno 104] Connection
[nltk_data]     reset by peer>
[nltk_data] Error loading omw-1.4: <urlopen error [Errno 104]
[nltk_data]     Connection reset by peer>


In [2]:
def getdata():
    path = '/home2/rudra.dhar/Data/SE/adr/data_folder/'

    directory = path + 'architecture-master'
    context, decision = [], []
    for filename in os.scandir(directory):
        #if filename.path.endswith(".md") and filename.is_file():
        with open(filename.path, 'r') as f:
            content = f.read()
            context.append(content[content.find('## Context')+12:content.find('## Decision')-2])
            decision.append(content[content.find('## Decision')+13:content.find('## Status')])
        

    directory = path + 'adr'
    #context, decision = [], []
    for filename in os.scandir(directory):
        #if filename.path.endswith(".md") and filename.is_file():
        with open(filename.path, 'r') as f:
            content = f.read()
            context.append(content[2:content.find('## Decision')-2])
            if ('## Pros and Cons' in content):
                decision.append(content[content.find('## Decision')+21:content.find('## Pros and Cons')])
            elif ('## License' in content):
                decision.append(content[content.find('## Decision')+21:content.find('## License')])
            
        

    directory = path + 'examples_extracted'
    #context, decision = [], []
    for filename in os.scandir(directory):
        with open(filename.path, 'r') as f:
            content = f.read()
            context.append(content[12:content.find('## Decision')-2])
            decision.append(content[content.find('## Decision')+13:])
        

    directory = path + 'cardano'
    #context, decision = [], []
    for filename in os.scandir(directory):
        with open(filename.path, 'r', encoding='utf-8') as f:
            content = f.read()
            context.append(content[12:content.find('## Decision')-2])
            decision.append(content[content.find('## Decision')+17:-2])
        

    directory = path + 'island'
    #context, decision = [], []
    for filename in os.scandir(directory):
        with open(filename, "r", encoding='utf-8') as f:
            content = f.read()
            if('\nConsidered Options' in content):
                context.append(content[content.find('Context and Problem Statement')+30:content.find('\nConsidered Options')])
            elif('\nDecision Drivers' in content):
                context.append(content[content.find('Context and Problem Statement')+30:content.find('\nDecision Drivers')])
            else:
                context.append(content[content.find('Context and Problem Statement')+30:content.find('\nDecision Outcome')])

            if('\nPros and Cons of ' in content):
                decision.append(content[content.find('Decision Outcome')+17:content.find('\nPros and Cons of ')])
            else:
                decision.append(content[content.find('Decision Outcome')+17:])

        
    context_input_gpt3 = ["Architectural Decision Record\n\n## Context\n" +c+ "\n\n## Decision\n" for c in context]
    context_input_chatGPT = ["## Context\n\n" +c for c in context]
    context_input_T5 = ["This is a Architectural Decision Record. Provide a Decision for the Context given below.\n\n## Context\n" +c for c in context]
    
    return context, decision, context_input_gpt3, context_input_chatGPT, context_input_T5


In [13]:
# gpt_3
context, decision, context_input, _ = getdata()

random_index = np.random.randint(len(context_input), size=(5))
context = [context[i] for i in random_index]
decision = [decision[i] for i in random_index]
context_input = [context_input[i] for i in random_index]
print(len(context_input))

5


In [5]:
# gpt_4
context, decision, _, context_input = getdata()

random_index = np.random.randint(len(context_input), size=(5))
context = [context[i] for i in random_index]
decision = [decision[i] for i in random_index]
context_input = [context_input[i] for i in random_index]
print(len(context_input))

5


In [3]:
# T5
_, decision, _, _, context = getdata()

random_index = np.random.randint(len(context), size=(5))
context = [context[i] for i in random_index]
decision = [decision[i] for i in random_index]
print(len(context))

5


### Pipeline Gpt2

In [21]:
from transformers import pipeline

In [48]:
def gpt_2(context, decision, model):

    # Total length of tokens should be less than 1024
    # approx less than 512 tokens for context and 512 tokens for decision
    # approx 512 tokens = 2000 characters
    context_truncated = [c[:2000]+"\n\n## Decision" for c in context]
    decision_reference_truncated = [c[:2000] for c in decision]

    generator = pipeline('text-generation', model=model, device = 0)

    prediction = generator(context_truncated, max_length=1000, num_return_sequences=1)
    prediction = [p[0]['generated_text'] for p in prediction]

    predicted_decision = []
    for p in range(len(prediction)):
        predicted_decision.append(prediction[p][len(context_truncated[p]):])

    results = rouge.compute(predictions=predicted_decision, references=decision_reference_truncated)
    ic(results)
    results = bleu.compute(predictions=predicted_decision, references=decision_reference_truncated)
    ic(results)
    results = meteor.compute(predictions=predicted_decision, references=decision_reference_truncated)
    ic(results)
    results = bertscore.compute(predictions=predicted_decision, references=decision_reference_truncated, lang="en")
    results = {
        'precision': np.average(results['precision']), 'recall': np.average(results['recall']),
        'f1': np.average(results['f1']), 'hashcode': results['hashcode']
        }
    ic(results)

In [50]:
for model in ['gpt2']:
    ic('#############################################\n',model)
    gpt_2(context, decision, model)

############################################# gpt2


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'rouge1': 0.2827944942381562, 'rouge2': 0.023629997207363966, 'rougeL': 0.13010563380281692, 'rougeLsum': 0.25695689287238577}
{'bleu': 0.0, 'precisions': [0.23645833333333333, 0.027139874739039668, 0.0020920502092050207, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 1.6080402010050252, 'translation_length': 960, 'reference_length': 597}
{'meteor': 0.21987588412929074}
{'precision': 0.804944634437561, 'recall': 0.80194091796875, 'f1': 0.8033516108989716, 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.24.0)'}


### Api GPT3-4

In [35]:
import openai
from dotenv import load_dotenv
path = '/home2/rudra.dhar/codes/SE/adr/'
load_dotenv(path + '.env')
openai.api_key = os.getenv("OPENAI_API_KEY")

In [36]:
def askGPT3(text, model, max_tokens):    
    response = openai.Completion.create(
        model=model,
        prompt=text,
        max_tokens=max_tokens,
    )
    return response.choices[0].text

def gpt_3(context, decision, model, max_tokens):

    # Total length of tokens should be less than 2048 for ada, 4096 for text-davinci-003
    # approx half for context and half for decision
    # approx 1000 tokens = 4000 characters

    max_char = max_tokens*4
    context_truncated = [c[:max_char] for c in context]
    decision_reference_truncated = [c[:max_char] for c in decision]

    predicted_decision = []
    for c in context_truncated:
        predicted_decision.append(askGPT3(c, model, max_tokens))
    
    for i in range(len(predicted_decision)):
        if('## Decision' in predicted_decision[i]):
            predicted_decision[i] = predicted_decision[i][predicted_decision[i].find('## Decision')+12:]
            predicted_decision[i] = predicted_decision[i][:predicted_decision[i].find('\n## ')]

    results = rouge.compute(predictions=predicted_decision, references=decision_reference_truncated)
    ic(results)
    results = bleu.compute(predictions=predicted_decision, references=decision_reference_truncated)
    ic(results)
    results = meteor.compute(predictions=predicted_decision, references=decision_reference_truncated)
    ic(results)
    results = bertscore.compute(predictions=predicted_decision, references=decision_reference_truncated, lang="en")
    results = {
        'precision': np.average(results['precision']), 'recall': np.average(results['recall']),
        'f1': np.average(results['f1']), 'hashcode': results['hashcode']
        }
    ic(results)

In [37]:
context, decision, context_input, _ = getdata()
random_index = np.random.randint(len(context_input), size=(5))
context = [context[i] for i in random_index]
decision = [decision[i] for i in random_index]
context_input = [context_input[i] for i in random_index]
print(len(context_input))

for m in [('ada', 1000), ('text-davinci-003', 2000)]:
    model, max_tokens = m
    ic('#############################################\n',model)
    gpt_3(context_input, decision, model, max_tokens)

ic| '#############################################\n': '#############################################
                                                        '
    model: 'ada'
    max_tokens: 1000


5


ic| results: {'rouge1': 0.2084249540892277,
              'rouge2': 0.023471263998767988,
              'rougeL': 0.10763358254162425,
              'rougeLsum': 0.16692793878168832}
ic| results: {'bleu': 0.0,
              'brevity_penalty': 1.0,
              'length_ratio': 2.019582245430809,
              'precisions': [0.17711700064641242,
                             0.02529182879377432,
                             0.001951854261548471,
                             0.0],
              'reference_length': 766,
              'translation_length': 1547}
ic| results: {'meteor': 0.19089581761534208}
ic| results: {'f1': 0.8133254528045655,
              'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.24.0)',
              'precision': 0.8014590024948121,
              'recall': 0.8261906743049622}
ic| '#############################################\n': '#############################################
                                                        '
    model: 't

In [43]:
def ask_chatGPT(text, model, max_tokens):    
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {
            "role": "system",
            "content": "This is an Architectural Decision Record. Give a ## Decision corresponding to the ## Context provided by the user"
            },
            {
            "role": "user",
            "content": text
            }
        ],
        max_tokens=max_tokens,
    )
    return response.choices[0].message.content

def chatGPT(context, decision, model, max_tokens):
    max_char = max_tokens*4
    context_truncated = [c[:max_char] for c in context]
    decision_reference_truncated = [c[:max_char] for c in decision]

    predicted_decision = []
    for c in context_truncated:
        if(model == 'gpt-4'):
            time.sleep(30)
        predicted_decision.append(ask_chatGPT(c, model, max_tokens))

    for i in range(len(predicted_decision)):
        if('## Decision' in predicted_decision[i]):
            predicted_decision[i] = predicted_decision[i][predicted_decision[i].find('## Decision')+12:]
            predicted_decision[i] = predicted_decision[i][:predicted_decision[i].find('\n## ')]

    
    results = rouge.compute(predictions=predicted_decision, references=decision_reference_truncated)
    ic(results)
    results = bleu.compute(predictions=predicted_decision, references=decision_reference_truncated)
    ic(results)
    results = meteor.compute(predictions=predicted_decision, references=decision_reference_truncated)
    ic(results)
    results = bertscore.compute(predictions=predicted_decision, references=decision_reference_truncated, lang="en")
    results = {
        'precision': np.average(results['precision']), 'recall': np.average(results['recall']),
        'f1': np.average(results['f1']), 'hashcode': results['hashcode']
        }
    ic(results)

In [45]:
context, decision, _, context_input = getdata()
random_index = np.random.randint(len(context_input), size=(5))
context = [context[i] for i in random_index]
decision = [decision[i] for i in random_index]
context_input = [context_input[i] for i in random_index]
print(len(context_input))

for m in [('gpt-3.5-turbo', 2000), ('gpt-4', 4000)]:
    model, max_tokens = m
    print('#############################################\n',model)
    chatGPT(context_input, decision, model, max_tokens)

5
#############################################
 gpt-4


ic| results: {'rouge1': 0.2361695019093232,
              'rouge2': 0.08940728444487843,
              'rougeL': 0.17369755919904878,
              'rougeLsum': 0.18926912390268893}
ic| results: {'bleu': 0.04269381380644307,
              'brevity_penalty': 1.0,
              'length_ratio': 1.6458333333333333,
              'precisions': [0.19746835443037974,
                             0.041025641025641026,
                             0.025974025974025976,
                             0.015789473684210527],
              'reference_length': 240,
              'translation_length': 395}
ic| results: {'meteor': 0.29025597520096386}
ic| results: {'f1': 0.862637209892273,
              'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.24.0)',
              'precision': 0.8516870021820069,
              'recall': 0.8746695756912232}


### T5

In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [11]:
def ask_t5(context, decision, model_name):
    tokenizer = T5Tokenizer.from_pretrained(model_name, cache_dir='/scratch/rudra.dhar/cache', model_max_length=4096)
    model = T5ForConditionalGeneration.from_pretrained(model_name, cache_dir='/scratch/rudra.dhar/cache', device_map="balanced")

    predicted_decision = []
    
    for c in context:
        input_ids = tokenizer(c, return_tensors="pt").input_ids.cuda()
        #print(len(input_ids[0]))
        outputs = model.generate(input_ids, max_length=len(input_ids[0])*4, min_length= int(len(input_ids[0])/8))
        predicted_decision.append(tokenizer.decode(outputs[0], skip_special_tokens=True))

    results = rouge.compute(predictions=predicted_decision, references=decision)
    ic(results)
    results = bleu.compute(predictions=predicted_decision, references=decision)
    ic(results)
    results = meteor.compute(predictions=predicted_decision, references=decision)
    ic(results)
    results = bertscore.compute(predictions=predicted_decision, references=decision, lang="en")
    results = {
        'precision': np.average(results['precision']), 'recall': np.average(results['recall']),
        'f1': np.average(results['f1']), 'hashcode': results['hashcode']
        }
    ic(results)

    model_name = model_name.replace('/', '_')
    df = pd.DataFrame(list(zip(context, decision, predicted_decision)), columns =['context', 'decision', 'predicted_decision'])
    df.to_csv('/home2/rudra.dhar/Data/SE/adr/output/'+model_name+'.csv', index=False)
    del tokenizer, model, input_ids, outputs, predicted_decision, df
    torch.cuda.empty_cache()

In [10]:
#_, decision, _, _, context = getdata()
t5_models = ['t5-small', 't5-base', 't5-large', 't5-3b', 'bigscience/T0_3B',
             'google/flan-t5-small','google/flan-t5-base', 'google/flan-t5-large', 'google/flan-t5-xl']
t5_models = ['google/flan-t5-base', 'google/flan-t5-large', 'google/flan-t5-xl']
for model in t5_models:
    ic('#############################################\n',model)
    ask_t5(context, decision, model)

ic| '#############################################\n': '#############################################
                                                        '
    model: 'google/flan-t5-xl'
ic| results: {'rouge1': 0.20977577029170852,
              'rouge2': 0.1359309406440554,
              'rougeL': 0.2050271669397532,
              'rougeLsum': 0.2092171110738314}
ic| results: {'bleu': 0.0012491881646322443,
              'brevity_penalty': 0.043664418982504526,
              'length_ratio': 0.2420591456736035,
              'precisions': [0.17647058823529413,
                             0.038901601830663615,
                             0.013888888888888888,
                             0.00702576112412178],
              'reference_length': 1826,
              'translation_length': 442}
ic| results: {'meteor': 0.16364070273487932}
ic| results: {'f1': 0.8253636837005616,
              'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.24.0)',
              'precisio

In [8]:
torch.cuda.empty_cache()

### Llama2

In [56]:
from transformers import pipeline

generator = pipeline('text-generation', model='gpt2', device_map="auto", model_kwargs={"cache_dir": "/scratch/rudra.dhar/cache/"})

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
    model_kwargs={"cache_dir": "/scratch/rudra.dhar/cache/"},
)

sequences = pipeline(
    'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")
